In [1]:
from bs4 import BeautifulSoup
import csv
import re
import os 
import pandas as pd
import glob

In [2]:
data = []

# reading html files from folder

#'Riv_Shelter_HTML_Files'is the name for the shelter HTML Files Riverside
#'Riv_Food_Pantry_HTML_Files'is the name for the food pantry HTML Files for Riverside 
#'Riv_Mental_Health_HTML_Files'is the name for the shelter HTML Files for Riverside
#'Downtown_LA_Food_Pantry_HTML_Files' is the name for the food pantry HTML Files for downtown LA
folder_path = 'Irvine_Mental_Health_HTML_Files'

# Loop through each file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.html'):
        file_path = os.path.join(folder_path, filename)

        with open(file_path, 'r', encoding='utf-8') as file:
            html_content = file.read()

        soup = BeautifulSoup(html_content, 'html.parser')


        # Find the <ul> element with class "best-programs"
        ul_element = soup.find('ul', {'class': 'best-programs'})

        # Find all <li> elements within the <ul>
        li_elements = ul_element.find_all('li', {'class': 'search-result card card-v3 program-info'})

        # Loop through the found <li> elements and extract labels and values
        for li_element in li_elements:
            # Use BeautifulSoup to parse the HTML of each li_element
            li_soup = BeautifulSoup(str(li_element), 'html.parser')

            # Find the program heading 
            program_heading_element = li_soup.find('div', {'class': 'card-heading'})
            Service_name = program_heading_element.find('a', {'class': 'activity-log ph-flyout-click cwdc-flyout-click click-cookie'}).text
            print('Service name: ', Service_name)
    
            # Extract program URL
            program_url_element = program_heading_element.find('a', {'class': 'activity-log ph-flyout-click cwdc-flyout-click click-cookie'})
            program_url = program_url_element['href'] if program_url_element else None
            Service_url = "https://www.auntbertha.com/"+program_url
            print('URL: ', Service_url)
            
            is_reviewed = False

            # Reviewed on
            Reviewd_on_element = program_heading_element.find('div', {'class': 'last-reviewed'})
            if Reviewd_on_element:  # Check if the element exists
                Reviewd_on_text = Reviewd_on_element.get_text(strip=True)  # Clean up the text
                match = re.search(r'\d{2}/\d{2}/\d{4}', Reviewd_on_text)
                Reviewd_on = match.group() if match else None
                if Reviewd_on:  # If we found a review date
                    is_reviewed = True  # Set the flag to True
                print("Reviewed on:", Reviewd_on)
            else:
                Reviewd_on = None  # Set to None if not found
                print("No review date found.")
    
            # Access class="program-tags"
            program_tags = li_soup.find('div', {'class': 'program-tags'})

            # Main Services
            main_s = []
            main_service_list = program_tags.find('ul', {'class': 'list-inline'})
            main_service_items = main_service_list.find_all('li')

            for item in main_service_items:
                main_service = item.find('a', {'class': 'loading-on-click activity-log'}).text.strip()
                main_s.append(main_service)

            print("Main Services:", main_s)

            # Try to access the "Other Services" section
            other_service_list = program_tags.find('div', {'class': 'secondary-tags'})
            if other_service_list:
                other_s = []
                other_service_items = other_service_list.find('ul', {'class': 'list-inline'}).find_all('li')

                for item in other_service_items:
                    other_service = item.find('a', {'class': 'loading-on-click activity-log'}).text.strip()
                    other_s.append(other_service)
            else:
                other_s = None

            print("Other Services:", other_s)

            # Access the "Serving" section
            serving_section = program_tags.find('div', {'class': 'attribute-tags'})

            # Initialize a list to store the serving information
            serving_ = []
    
            # Find all the <li> elements within the serving section
            serving_items = serving_section.find_all('li')

            # Iterate through the serving items and extract the text from the <a> elements
            for item in serving_items:
                link = item.find('a', {'class': 'loading-on-click activity-log'})
    
                # Check if the link was found
                if link:  # Only proceed if the link exists
                    serving_text = link.text.strip()
                    serving_.append(serving_text)

            print("Serving:", serving_)

    
            # accessing next-steps-module, extract phone number, location, hours 
    
            next_steps_module = li_soup.find('div', {'class': 'next-steps-module'})

            # Extract phone number
            phone_number_elements = next_steps_module.find_all('span', {'class': 'result-next-step-item'})

            # Also find all 'a' elements with href attributes containing 'tel:'
            tel_link_elements = next_steps_module.find_all('a', href=True)

            phone_number = None  # Initialize phone number as None

            # Extract phone numbers from text-based spans
            for element in phone_number_elements:
                phone_number_text = element.text.strip() if element else None
    
                # Regex to extract digits (handling separators like spaces or hyphens)
                phone_number_matches = re.findall(r'[\d-]+', phone_number_text)
    
                # Join the digits into a single phone number string if matches are found
                if phone_number_matches:
                    phone_number = ''.join(phone_number_matches)
                    break  # Stop once we've found a phone number

            # If no phone number found from text, check the 'tel:' href links
            if not phone_number:
                for element in tel_link_elements:
                    href_value = element['href']
        
                    # Check if the href contains 'tel:' and extract digits
                    if 'tel:' in href_value:
                        phone_number_matches = re.findall(r'[\d-]+', href_value)
                        if phone_number_matches:
                            phone_number = ''.join(phone_number_matches)
                            break  # Stop once we've found a phone number

            # Print the extracted phone number
            if phone_number:
                print("Phone Number:", phone_number)
            else:
                print("No phone number found.")

            # Extract location address
            location_address_element = next_steps_module.find('a', {'class': 'activity-log ph-flyout-click cwdc-flyout-click map-link with-address'})
            location_address = location_address_element.text.strip() if location_address_element else None
            location_address = re.sub(r'\s+', ' ', location_address) if location_address_element else None
            print("Location Address:", location_address)

            # Extract URL
            location_url_map = location_address_element['href'] if location_address_element else None
            print("Location url map:", location_url_map)
            
            hours_info = {
                '24_hour': False,
                'Monday': None,
                'Tuesday': None,
                'Wednesday': None,
                'Thursday': None,
                'Friday': None,
                'Saturday': None,
                'Sunday': None
            }
            
            # Extract hours based on structure
            hours_element = next_steps_module.find('div', {'class': 'office-hours-schedule see-hours-dropdown'})
            if hours_element:
                # Find all the <span> elements within the hours_element
                day_spans = hours_element.find_all('span')

                # Ensure that we have even pairs of day and corresponding hours
                if len(day_spans) % 2 == 0:
                    # Iterate through the spans in pairs (day and its corresponding hours)
                    for i in range(0, len(day_spans), 2):
                        day_span = day_spans[i]          # This is the day (e.g., "Monday:")
                        hours_span = day_spans[i + 1]    # This is the corresponding hours (e.g., "Closed" or time)

                        # Extract the day name and remove the colon
                        day = day_span.text.strip()[:-1]  # Remove the ':' at the end
                        full_day = day.strip()            # Ensure there's no extra space

                        # Check if the day is valid and exists in hours_info
                        if full_day in hours_info:
                            # Check if the corresponding hours span indicates "Closed"
                            if 'Closed' in hours_span.text:
                                hours_info[full_day] = 'Closed'
                            else:
                                # Extract hours for non-closed days
                                hours_info[full_day] = hours_span.text.strip()
                else:
                    print("Unexpected hours format. Ensure that each day has corresponding hours.")

            # Handle 24-hour information if it exists
            else:
                hours_element = next_steps_module.find('span', {'class': 'result-geo-hours'})
                if hours_element:
                    # Get 24-hour text strip
                    hours_text = hours_element.get_text(strip=True)

                    # Assuming the presence of this text means 24-hour operation
                    if '24' in hours_text or '24-hour' in hours_text.lower():
                        hours_info['24_hour'] = True
                    else:
                        print("Unable to determine if the office is 24 hours.")
                else:
                    print("Hours information not available.")

            # Output the result
            print(hours_info)
    
    
            # Loop through the found <li> elements and extract labels and values
            Extra_element = li_element.find('div', {'class': 'panel-wrapper more-info-panel'})
            elig = []
            eligibility_rules_element = Extra_element.find('div', {'class': 'eligibility-rules'})
    
            # Check if eligibility_rules_element is found
            if eligibility_rules_element:
                # Check if eligibility_rules_element contains a list (ul)
                ul_element = eligibility_rules_element.find('ul')
        
                if ul_element:
                    # If it contains a list, extract list items and store them in a flat list
                    eligibility_list = [li.text.strip() for li in ul_element.find_all('li')]
                    elig.extend(eligibility_list)  # Use extend to add elements to the list directly
                    #print("Eligibility:", eligibility_list)
                else:
                    # If it doesn't contain a list, store the text as is
                    eligibility_text = eligibility_rules_element.text.strip()
                    #print("Eligibility:", [eligibility_text])  # Wrap in a list to maintain consistency
                    elig.append(eligibility_text)

            else:
                # Skip if 'eligibility-rules' class is not found
                pass
    
        
            print("Eligibility:", elig)
    
            # Extract Availability
            availability_element = Extra_element.find('strong', {'data-translate': 'Availability'})
            
            if availability_element:
                availability = availability_element.find_next('div', {'class': 'col-md-10'}).text.strip()
            else:
                availability = "Not specified"

            # Extract Description
            description_element = Extra_element.find('strong', {'data-translate': 'Description'})
            description = description_element.find_next('div', {'class': 'col-md-10'}).text.strip()

            # Extract Languages
            languages_element = Extra_element.find('strong', {'data-translate': 'Languages'})
            languages = languages_element.find_next('div', {'class': 'col-md-10'}).text.strip()
            languages_data = [lang.strip() for lang in languages.split(',')]
            
            # Extract Cost
            cost_element = Extra_element.find('strong', string='Cost:')
            
            if cost_element:  # Check if cost_element exists
                cost = cost_element.find_next('div', {'class': 'col-md-10'}).text.strip()
            else:
                cost = "Not specified"  # Default value if cost_element is not found
            
            # Extract Website URLs if they exist, or set them to None
            website_element = Extra_element.find('div', {'data-translation': 'Website'})
            website_url = website_element.find_next('a', {'class': 'activity-log descriptionProgramWebsite'})['href'] if website_element else None

            # Extract Facebook and Twitter URLs if they exist, or set them to None
            facebook_element = Extra_element.find('strong', {'data-translate': 'Facebook'})
            facebook_url = facebook_element.find_next('a', {'class': 'activity-log descriptionProgramFacebook'})['href'] if facebook_element else None

            twitter_element = Extra_element.find('strong', {'data-translate': 'Twitter'})
            twitter_url = twitter_element.find_next('a', {'class': 'activity-log descriptionProgramTwitter'})['href'] if twitter_element else None

            # Extract Coverage Area
            coverage_element = Extra_element.find('strong', {'data-translate': 'Coverage Area'})
            coverage = coverage_element.find_next('div', {'class': 'col-md-10'}).text.strip()
            
            # Initialize latitude and longitude as None by default
            latitude = None
            longitude = None
            zipcode = None

            # Find the element with class "office-hour-address"
            location_element = Extra_element.find('div', {'class': 'office-hours-address _js_address address notranslate'})

            # Check if the element exists and has the required attributes
            if location_element:
                latitude = location_element['data-latitude'] if location_element.has_attr('data-latitude') else None
                longitude = location_element['data-longitude'] if location_element.has_attr('data-longitude') else None
                print("Latitude:", latitude)
                print("Longitude:", longitude)
                
                # Extract all text within location_element
                address_text = location_element.get_text(separator=" ").strip()  # Get all text as a single string

                # Use regular expression to search for the ZIP code pattern anywhere in the text
                zip_matches = re.findall(r'(?<!\d)(\b\d{5}\b)(?!\d)', address_text)
                zipcode = int(zip_matches[-1]) if zip_matches else None
                print("ZIP Code:", zipcode)

            # Print or use the extracted values as needed
            print("Availability:", availability)
            print("Description:", description)
            print("Languages:", languages_data)
            print("Cost:", cost)
            print("Facebook URL:", facebook_url)
            print("Twitter URL:", twitter_url)
            print("Coverage Area:", coverage)

            data.append([
            Service_name,
            Service_url,
            main_s,
            other_s,
            serving_,
            phone_number,
            website_url,
            location_address,
            location_url_map,
            elig,
            availability,
            description,
            languages_data,
            cost,
            is_reviewed,
            facebook_url,
            twitter_url,
            coverage,
            latitude,
            longitude,
            zipcode,
            hours_info['24_hour'],
            hours_info['Monday'],      
            hours_info['Tuesday'],     
            hours_info['Wednesday'],   
            hours_info['Thursday'],   
            hours_info['Friday'],     
            hours_info['Saturday'],  
            hours_info['Sunday']])    
            print("***************************************************************************")

Service name:  Community Counseling
URL:  https://www.auntbertha.com//tarzana-treatment-centers%252C-inc.-%2528ttc%2529--long-beach-ca--community-counseling/5511145607659520?postal=92620
Reviewed on: 12/16/2024
Main Services: ['sober living', 'detox', 'residential treatment', 'family counseling', 'individual counseling']
Other Services: ['sober living', 'detox', 'outpatient treatment', 'residential treatment', 'prevent & treat', 'hiv treatment', 'mental health care', 'family counseling', 'after school care']
Serving: ['anyone in need', 'all ages', 'mentally incapacitated', 'individuals', 'families', 'spouses', 'depression']
Phone Number: 888-777-8565
Location Address: 2101 Magnolia Ave, Long Beach, CA 90806
Location url map: https://www.google.com/maps/?q=2101+Magnolia+Ave,+Long+Beach,+CA+90806/
{'24_hour': False, 'Monday': '8:00 AM - 5:00 PM  PST', 'Tuesday': '8:00 AM - 5:00 PM  PST', 'Wednesday': '8:00 AM - 5:00 PM  PST', 'Thursday': '8:00 AM - 5:00 PM  PST', 'Friday': '8:00 AM - 5:0

Service name:  Early Intervention Program
URL:  https://www.auntbertha.com//assistance-league-of-laguna-beach--laguna-beach-ca--early-intervention-program/4935192972361728?postal=92620
Reviewed on: 12/14/2024
Main Services: ['parenting education', 'group therapy', 'one-on-one support', 'support groups', 'youth development']
Other Services: None
Serving: ['infants', 'developmental disability', 'families', 'parents']
Phone Number: 949-494-6097
Location Address: 547 Catalina Street, Laguna Beach, CA 92651
Location url map: https://www.google.com/maps/?q=547+Catalina+Street,+Laguna+Beach,+CA+92651/
{'24_hour': False, 'Monday': 'Closed', 'Tuesday': 'Closed', 'Wednesday': '9:00 AM - 12:00 PM  PST', 'Thursday': 'Closed', 'Friday': 'Closed', 'Saturday': 'Closed', 'Sunday': 'Closed'}
Eligibility: ['This program helps people who are 0 to 1 years old.']
Latitude: 33.5412825
Longitude: -117.780513
ZIP Code: 92651
Availability: available
Description: Assistance League of Laguna Beach offers an Earl

Service name:  Warrior & Family Groups
URL:  https://www.auntbertha.com//ptsd-foundation-of-america--houston-tx--warrior-%2526-family-groups/5534962649530368?postal=92620
Reviewed on: 12/16/2024
Main Services: ['mental health care', 'support network', 'support groups']
Other Services: None
Serving: ['adults 18+', 'veterans', 'ptsd']
No phone number found.
Location Address: None
Location url map: None
{'24_hour': False, 'Monday': '6:00 AM - 3:00 PM  PST', 'Tuesday': '6:00 AM - 3:00 PM  PST', 'Wednesday': '6:00 AM - 3:00 PM  PST', 'Thursday': '6:00 AM - 3:00 PM  PST', 'Friday': '6:00 AM - 3:00 PM  PST', 'Saturday': 'Closed', 'Sunday': 'Closed'}
Eligibility: ['This program serves veterans dealing with PTSD.']
Latitude: 29.9234965
Longitude: -95.5448322
ZIP Code: 77064
Availability: available
Description: Warrior & Family Groups, also referred to as Combat Trauma Groups, are fellowships for combat veterans and their families to share their experiences, testimonies of healing, compassion, a

Service name:  Virtual Counseling
URL:  https://www.auntbertha.com//family-assessment-counseling-%2526-education-services-%2528faces%2529--santa-ana-ca--virtual-counseling/6581906752798720?postal=92620
Reviewed on: 11/13/2024
Main Services: ['counseling', 'mental health care', 'family counseling', 'individual counseling']
Other Services: None
Serving: ['adults', 'young adults', 'teens', 'school-aged children', 'preschoolers', 'toddlers', 'seniors', 'individuals', 'families', 'low-income', 'uninsured', 'all mental health']
Phone Number: 714-447-9024
Location Address: None
Location url map: None
{'24_hour': False, 'Monday': 'Closed', 'Tuesday': 'Closed', 'Wednesday': 'Closed', 'Thursday': '9:00 AM - 5:00 PM  PST', 'Friday': '9:00 AM - 5:00 PM  PST', 'Saturday': 'Closed', 'Sunday': 'Closed'}
Eligibility: ['Must meet income requirements for sliding scale fees.']
Availability: available
Description: FACES Virtual Counseling Services help people learn how to deal more effectively with situat

Service name:  Personalized Care for Mental Health and Learning Disorders
URL:  https://www.auntbertha.com//child-mind-institute%252C-inc.--new-york-ny--personalized-care-for-mental-health-and-learning-disorders/5889297183670272?postal=92620
Reviewed on: 12/17/2024
Main Services: ['outpatient treatment', 'disability screening', 'mental health care', 'group therapy', 'family counseling', 'individual counseling', 'mental health evaluation']
Other Services: None
Serving: ['young adults', 'teens', 'school-aged children', 'preschoolers', 'learning disability', 'add/adhd', 'developmental disability', 'autism', 'individuals', 'families', 'with children', 'low-income', 'limited english', 'trauma survivors', 'depression', 'bipolar', 'suicidal thoughts', 'anxiety']
Phone Number: 650-931-6565
Location Address: None
Location url map: None
{'24_hour': False, 'Monday': '8:00 AM - 8:00 PM  PST', 'Tuesday': '8:00 AM - 8:00 PM  PST', 'Wednesday': '8:00 AM - 8:00 PM  PST', 'Thursday': '8:00 AM - 8:00 PM

...

Service name:  Caregiver Support Program
URL:  https://www.auntbertha.com//santa-clarita-valley-senior-center--santa-clarita-ca--caregiver-support-program/5378252180291584?postal=92620
Reviewed on: 12/15/2024
Main Services: ['daily life skills', 'counseling', 'support groups', 'more education', 'skills & training']
Other Services: None
Serving: ['adults 18+', 'caregivers', 'depression']
Phone Number: 661-259-9444
Location Address: 22900 Market St, Santa Clarita, CA 91321
Location url map: https://www.google.com/maps/?q=22900+Market+St,+Santa+Clarita,+CA+91321/
{'24_hour': False, 'Monday': '8:00 AM - 5:00 PM  PST', 'Tuesday': '8:00 AM - 5:00 PM  PST', 'Wednesday': '8:00 AM - 5:00 PM  PST', 'Thursday': '8:00 AM - 5:00 PM  PST', 'Friday': '8:00 AM - 5:00 PM  PST', 'Saturday': 'Closed', 'Sunday': 'Closed'}
Eligibility: []
Latitude: 34.3753933
Longitude: -118.5294961
ZIP Code: 91321
Availability: available
Description: The Santa Clarita Valley Senior Center seeks to promote the quality of l

Service name:  Substance Use Disorder Treatment
URL:  https://www.auntbertha.com//korean-community-services-%2528kcs%2529--irvine-ca--substance-use-disorder-treatment/5363551430508544?postal=92620
Reviewed on: 12/15/2024
Main Services: ['addiction & recovery', 'outpatient treatment', 'substance abuse counseling', 'individual counseling', 'mental health evaluation', 'navigating the system', 'case management', 'support network']
Other Services: None
Serving: ['adults', 'young adults', 'seniors', 'individuals', 'families', 'low-income', 'uninsured', 'underinsured', 'limited english', 'asian', 'substance dependency']
Phone Number: 714-503-6550
Location Address: 19742 Macarthur Boulevard, Irvine, CA 92612
Location url map: https://www.google.com/maps/?q=19742+Macarthur+Boulevard,+Irvine,+CA+92612/
{'24_hour': False, 'Monday': '8:00 AM - 5:00 PM  PST', 'Tuesday': '8:00 AM - 5:00 PM  PST', 'Wednesday': '8:00 AM - 5:00 PM  PST', 'Thursday': '8:00 AM - 5:00 PM  PST', 'Friday': '8:00 AM - 5:00 P

Service name:  Inpatient Drug Rehab
URL:  https://www.auntbertha.com//anaheim-lighthouse--anaheim-ca--inpatient-drug-rehab/5237835664850944?postal=92620
Reviewed on: 12/15/2024
Main Services: ['addiction & recovery', 'detox', 'residential treatment']
Other Services: None
Serving: ['adults', 'young adults', 'teens', 'seniors', 'individuals', 'substance dependency', 'opioid dependency', 'alcohol dependency', 'dual diagnosis']
Phone Number: 877-959-5909
Location Address: 1320 West Pearl Street, Anaheim, CA 92801
Location url map: https://www.google.com/maps/?q=1320+West+Pearl+Street,+Anaheim,+CA+92801/
{'24_hour': True, 'Monday': None, 'Tuesday': None, 'Wednesday': None, 'Thursday': None, 'Friday': None, 'Saturday': None, 'Sunday': None}
Eligibility: ['This program serves people with alcohol or drug dependencies.']
Latitude: 33.8341825
Longitude: -117.9311703
ZIP Code: 92801
Availability: available
Description: Inpatient drug rehab offers a stable environment for learning new skills, atti

Service name:  Mental Health Services
URL:  https://www.auntbertha.com//american-living-organ-donor-fund-%2528alodf%2529--philadelphia-pa--mental-health-services/4533349973098496?postal=92620
Reviewed on: 12/12/2024
Main Services: ['mental health care', 'virtual support']
Other Services: None
Serving: ['all ages', 'all care phases', 'individuals', 'families', 'caregivers']
Phone Number: 215-601-6530
Location Address: None
Location url map: None
{'24_hour': False, 'Monday': '8:00 AM - 5:00 PM', 'Tuesday': '8:00 AM - 5:00 PM', 'Wednesday': '8:00 AM - 5:00 PM', 'Thursday': '8:00 AM - 5:00 PM', 'Friday': '8:00 AM - 5:00 PM', 'Saturday': 'Closed', 'Sunday': 'Closed'}
Eligibility: ['This program helps living organ donors.', 'Must be a US citizen.']
Availability: available
Description: American Living Organ Donor Fund provides free mental health services by trained providers to any living donor pre- and post-donation. This program provides:- Mental health care Mental health services are provi

Service name:  Community Counseling - Individual Counseling
URL:  https://www.auntbertha.com//mariposa-women-and-family-center--orange-ca--community-counseling---individual-counseling/5703027608518656?postal=92620
No review date found.
Main Services: ['individual counseling']
Other Services: ['counseling']
Serving: ['all ages', 'individuals', 'low-income']
Phone Number: 714-547-6494
Location Address: 812 W. Town & Country Road Bldg C, Orange, CA 92868
Location url map: https://www.google.com/maps/?q=812+W.+Town+&+Country+Road+Bldg+C,+Orange,+CA+92868/
{'24_hour': False, 'Monday': '8:00 AM - 5:00 PM  PST', 'Tuesday': '8:00 AM - 5:00 PM  PST', 'Wednesday': '8:00 AM - 5:00 PM  PST', 'Thursday': '8:00 AM - 5:00 PM  PST', 'Friday': '8:00 AM - 5:00 PM  PST', 'Saturday': 'Closed', 'Sunday': 'Closed'}
Eligibility: []
Latitude: 33.775816
Longitude: -117.8613746
ZIP Code: 92868
Availability: Not specified
Description: Individual Counseling helps you navigate the troubles you face, our counselors

In [3]:
# Save to CSV file 
#"FindHelp_extracted_data_riv_shelter.csv" is the name for the shelter csv for riv
#"FindHelp_extracted_data_riv_food_pantry.csv" is the name for the food pantry csv for riv
#"FindHelp_extracted_data_riv_mental_health.csv" is the name for the food pantry csv for riv
#"FindHelp_extracted_data_downtown_la_food_pantry.csv" is the name for the food pantry csv for downtown la
csv_filename = "FindHelp_extracted_data_irvine_mental_health.csv"

with open(csv_filename, 'w', newline='', encoding='utf-8') as csv_file:
    csv_writer = csv.writer(csv_file)
    
    # The header 
    header = [
        "Service_name",
        "Service_url",
        "Main_Services",
        "Other_Services",
        "Serving",
        "Phone_Number",
        "Website",
        "Location_Address",
        "Location_URL_Map",
        "Eligibility",
        "Availability",
        "Description",
        "Languages",
        "Cost",
        "Google_Review",
        "Facebook_URL",
        "Twitter_URL",
        "Coverage",
        "Latitude",
        "Longitude",
        "Zipcode",
        "24hour",
        "Monday",
        "Tuesday",
        "Wednesday",
        "Thursday",
        "Friday",
        "Saturday",
        "Sunday"
    ]
    csv_writer.writerow(header)
    
    csv_writer.writerows(data)

In [4]:
#"FindHelp_extracted_data_riv_shelter.csv" is the name for the shelter csv
#"FindHelp_extracted_data_riv_food_pantry.csv" is the name for the food pantry csv
#"FindHelp_extracted_data_riv_mental_health.csv" is the name for the food pantry csv
#"FindHelp_extracted_data_downtown_la_food_pantry.csv"
#"FindHelp_extracted_data_irvine_food_pantry.csv"
#"FindHelp_extracted_data_irvine_shelter.csv"
df = pd.read_csv("FindHelp_extracted_data_irvine_mental_health.csv")

#riv_zipcodes = [92501, 92502, 92503, 92504, 92505, 92506, 92507, 92508, 92513, 92514, 92516, 92517, 92521, 92522]

#downtown_la_zipcodes = [90012, 90013, 90014, 90015, 90017, 90021]

irvine_zipcodes = [92602, 92603, 92604, 92606, 92612, 92614, 92617, 92618, 92620]

filtered_df = df[df['Zipcode'].isin(irvine_zipcodes) | (df['Location_Address'].isna() & df['Longitude'].isna() & df['Latitude'].isna())]

filtered_df = filtered_df.drop_duplicates(subset = ['Website', 'Service_name'])

filtered_df.to_csv("FindHelp_extracted_data_irvine_mental_health.csv", index=False, encoding='utf-8')

filtered_df

,Service_name,Service_url,Main_Services,Other_Services,Serving,Phone_Number,Website,Location_Address,Location_URL_Map,Eligibility,...,Longitude,Zipcode,24hour,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
3,Yoga online via zoom,https://www.auntbertha.com//siloam-wellness--p...,"['health education', 'understand mental health...","['health education', 'recreation']","['anyone in need', 'adults 18+', 'hiv/aids', '...",215-765-6633,https://www.siloamwellness.org/,NaN,NaN,['All individuals 18 years of age and over.'],...,NaN,NaN,False,5:00 AM - 2:00 PM PST,Closed,Closed,Closed,Closed,Closed,Closed
5,Mental Health Support and Crisis Text Hotline,https://www.auntbertha.com//crisis-text-line--...,"['mental health care', 'help hotlines', 'one-o...",NaN,"['anyone in need', 'adults', 'young adults', '...",NaN,https://www.crisistextline.org/text-us/,NaN,NaN,['This program is available to anyone in need.'],...,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,ACA Health Enrollment: Low to No Cost Medical ...,https://www.auntbertha.com//care-guide-service...,"['discounted healthcare', 'health insurance', ...","['medical care', 'checkup & test', 'prevent & ...","['anyone in need', 'adults', 'young adults', '...",844-358-3286,https://careguideservices.com/,NaN,NaN,[],...,NaN,NaN,False,6:00 AM - 3:00 PM PST,6:00 AM - 3:00 PM PST,6:00 AM - 3:00 PM PST,6:00 AM - 3:00 PM PST,6:00 AM - 1:00 PM PST,Closed,Closed
7,Wellness Program,https://www.auntbertha.com//iprevail--chicago-...,"['understand mental health', 'mental health ca...",NaN,"['anyone in need', 'adults', 'young adults', '...",312-441-9137,https://www.iprevail.com/,NaN,NaN,['Cigna insurance carriers can access this pro...,...,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Hold Onto Hope,https://www.auntbertha.com//honor-connor--sims...,"['health education', 'understand mental health...",['bereavement'],"['anyone in need', 'adults', 'young adults', '...",NaN,https://honorconnor.org/hold-onto-hope/,NaN,NaN,['This program serves individuals who have per...,...,NaN,NaN,False,8:00 AM - 5:00 PM,8:00 AM - 5:00 PM,8:00 AM - 5:00 PM,8:00 AM - 5:00 PM,8:00 AM - 5:00 PM,Closed,Closed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
524,National Maternal Mental Health Hotline,https://www.auntbertha.com//health-resources-a...,"['understand mental health', 'mental health ca...",['translation & interpretation'],"['adults', 'young adults', 'teens', 'female', ...",833-852-6262,https://mchb.hrsa.gov/national-maternal-mental...,NaN,NaN,"['This program serves pregnant women, and new ...",...,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
529,Behavioral Health & Wellness,https://www.auntbertha.com//children%2527s-ins...,"['mental health care', 'counseling', 'family c...",NaN,"['all ages', 'families', 'with children', 'sin...",213-260-7600,https://www.childrensinstitute.org/parents/,NaN,NaN,[],...,NaN,NaN,False,8:00 AM - 5:00 PM PST,8:00 AM - 5:00 PM PST,8:00 AM - 5:00 PM PST,8:00 AM - 5:00 PM PST,8:00 AM - 5:00 PM PST,Closed,Closed
531,Lighthouse Educational Ministries,https://www.auntbertha.com//lighthouse-educati...,"['addiction & recovery', '12-step', 'daily lif...",NaN,"['adults', 'young adults', 'teens', 'seniors',...",949-599-4212,http://www.lighthouseedministries.org,NaN,NaN,[],...,NaN,NaN,False,8:00 AM - 5:00 PM PST,8:00 AM - 5:00 PM PST,8:00 AM - 5:00 PM PST,8:00 AM - 5:00 PM PST,8:00 AM - 5:00 PM PST,Closed,Closed
533,Journey to Remember Program,https://www.auntbertha.com//forever-footprints...,"['parenting education', 'checkup & test', 'end...",NaN,"['children', 'grieving']",714-509-0065,http://www.foreverfootprints.org/programs/jour...,NaN,NaN,[],...,NaN,NaN,False,8:00 AM - 5:00 PM PST,8:00 AM - 5:00 PM PST,8:00 AM - 5:00 PM PST,8:00 AM - 5:00 PM PST,8:00 AM - 5:00 PM PST,Closed,Closed
